In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [14]:
data1 = pd.read_csv("../../dataset/NASA/PC5.csv")
data1.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DESIGN_COMPLEXITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,0,1,0,0,0,0,1,1.00,0,1,...,2,1.00,0,1,0,1,1,0.00,0,N
1,8,5,9,0,6,8,3,0.19,4,1,...,24,0.09,33,58,14,14,34,27.27,16,N
2,3,1,4,0,3,0,1,0.17,0,1,...,6,0.07,11,18,7,9,15,33.33,6,N
3,0,1,1,0,0,0,1,0.33,0,1,...,3,0.14,4,9,4,8,7,0.00,3,N
4,3,25,0,18,9,40,13,0.19,16,1,...,32,0.15,108,134,18,19,86,35.06,68,N


In [15]:
data1.describe()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DESIGN_COMPLEXITY,...,MULTIPLE_CONDITION_COUNT,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL
count,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,...,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000,16962.000000
mean,1.366054,2.883917,0.631293,0.481016,1.681936,3.130527,1.975003,0.542105,1.327320,1.376548,...,1.629820,5.965511,0.286844,14.835397,24.129230,4.521047,4.484141,14.838816,4.229380,9.521519
std,12.822247,14.466359,2.212278,3.085064,16.646452,24.010081,7.420118,0.251830,10.183716,3.303685,...,12.839782,27.633131,0.202635,127.164295,210.937326,26.097461,4.279696,87.777585,12.389354,62.775792
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,1.000000,...,0.000000,2.000000,0.200000,0.000000,3.000000,0.000000,3.000000,4.000000,0.000000,2.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,1.000000,...,0.000000,2.000000,0.250000,1.000000,3.000000,1.000000,3.000000,4.000000,0.000000,2.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.000000,1.000000,...,0.000000,3.000000,0.250000,3.000000,7.000000,3.000000,6.000000,5.000000,0.000000,2.000000
max,704.000000,731.000000,69.000000,180.000000,901.000000,1090.000000,366.000000,1.500000,448.000000,223.000000,...,578.000000,1078.000000,1.000000,5169.000000,10862.000000,2241.000000,85.000000,3025.000000,94.740000,2072.000000


In [16]:
data1.values.shape

(16962, 39)

In [17]:
data2=data1.values
for i in range(16962):
    if data2[i][38]=="Y":
        data2[i][38]=1
    else:
        data2[i][38]=0
data2[1]

array([8, 5, 9, 0, 6, 8, 3, 0.19, 4, 1, 0.33, 25, 1, 0.0, 16, 0, 1, 0.33,
       26.51, 16.5, 7218.24, 0.15, 91, 0.06, 401.01, 437.47, 0.33, 2, 4,
       24, 0.09, 33, 58, 14, 14, 34, 27.27, 16, 0], dtype=object)

In [18]:
X=np.array(data2).astype(float)  

In [19]:
X[:,:38]

array([[ 0.  ,  1.  ,  0.  , ...,  1.  ,  0.  ,  0.  ],
       [ 8.  ,  5.  ,  9.  , ..., 34.  , 27.27, 16.  ],
       [ 3.  ,  1.  ,  4.  , ..., 15.  , 33.33,  6.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ...,  8.  ,  0.  ,  6.  ],
       [ 1.  ,  1.  ,  4.  , ..., 13.  , 30.  , 10.  ],
       [10.  , 17.  ,  8.  , ..., 71.  , 20.34, 50.  ]])

In [20]:
y=X[:,38]

In [21]:
X=X[:,0:38]

In [22]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((32920, 38), (32920,))

In [23]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(32920, 38)

In [24]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime
import csv
import time

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [25]:
#将数据集划分为训练集和测试集
#traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.2,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [26]:
Evaluat_list=[]
for i in np.arange(0,10):
    #将数据集划分为训练集和测试集
    traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = i)
    bls = broadnet(maptimes = 15, 
               enhencetimes = 15,
               map_function = 'relu',
               enhence_function = 'relu',
               batchsize =100,
               reg = 0.001)

    #根据训练数据设置权重
    weight=bls.addWeight(trainlabel)

    #训练
    starttime = datetime.datetime.now()
    bls.fit(traindata,trainlabel,weight)
    endtime = datetime.datetime.now()
    trainingTime=(endtime - starttime).total_seconds()
    #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

    #预测
    predictlabel = bls.predict(testdata)

    #评价指标计算
    acc=accuracy_score(testlabel,predictlabel,normalize=True)
    fmeasure=f1_score(testlabel,predictlabel, average='weighted', labels=np.unique(testlabel))
    try:
        auc=roc_auc_score(testlabel,predictlabel, average='macro', sample_weight=None)
    except ValueError:
        pass
    MCC=matthews_corrcoef(testlabel,predictlabel)
    Gmeasure=geometric_mean_score(testlabel,predictlabel, average='weighted')
    recall=recall_score(testlabel, predictlabel, average='weighted')

    print('trainingTime：%f,acc：%f,fmeasure：%f,auc：%f,recall：%f,MCC：%f,Gmeasure：%f'%(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure))
    Evaluat_tuple=(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure)    
    Evaluat_list.append(Evaluat_tuple)

# 表头
header = ['trainTime', 'acc', 'fmeasure', 'auc','recall', 'MCC', 'Gmeasure']
with open('./data_remember/0.75WBLS_Smote_PC5_Evaluat.csv', 'w', encoding='utf-8', newline='') as file_obj:
    # 创建对象
    writer = csv.writer(file_obj)
    # 写表头
    writer.writerow(header)
    # 3.写入数据(一次性写入多行)
    writer.writerows(Evaluat_list)


    #sio.savemat('./data_remember/WBLS_Smote_CM1_test.mat',{'time':format((endtime - starttime).total_seconds()),'acc':acc,'f1':fmeasure,'auc':auc,'mcc':MCC,'Gm':Gmeasure})

trainingTime：21.545849,acc：0.975699,fmeasure：0.975695,auc：0.975621,recall：0.975699,MCC：0.951558,Gmeasure：0.975621
trainingTime：21.646728,acc：0.972783,fmeasure：0.972784,auc：0.972918,recall：0.972783,MCC：0.945729,Gmeasure：0.972918
trainingTime：21.795734,acc：0.975942,fmeasure：0.975942,auc：0.976018,recall：0.975942,MCC：0.952025,Gmeasure：0.976018
trainingTime：21.890066,acc：0.975456,fmeasure：0.975457,auc：0.975494,recall：0.975456,MCC：0.950909,Gmeasure：0.975494
trainingTime：22.154941,acc：0.977521,fmeasure：0.977517,auc：0.977515,recall：0.977521,MCC：0.955402,Gmeasure：0.977515
trainingTime：21.951037,acc：0.975699,fmeasure：0.975700,auc：0.975859,recall：0.975699,MCC：0.951619,Gmeasure：0.975859
trainingTime：22.091139,acc：0.975213,fmeasure：0.975213,auc：0.975359,recall：0.975213,MCC：0.950658,Gmeasure：0.975359
trainingTime：22.033871,acc：0.973876,fmeasure：0.973868,auc：0.973761,recall：0.973876,MCC：0.948094,Gmeasure：0.973761
trainingTime：22.250321,acc：0.975334,fmeasure：0.975328,auc：0.975253,recall：0.975334,MCC：0